<a href="https://colab.research.google.com/github/SheidaTalei/SampleQandA/blob/main/Sample_Q%26A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
nb_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0,nb_path)

Mounted at /content/drive


In [ ]:
#!pip install --target=$nb_path hazm
# !pip install --target=$nb_path persianutils

In [41]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
#Convert a collection of raw documents to a matrix of TF-IDF features.
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import regex
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from hazm import *
import re
import string
import persianutils as pu
import persianutils.PersianAlphabet as pALpha

In [42]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import MaxPooling1D
from keras.layers import Conv1D


from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.utils import class_weight

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

from sklearn.metrics import  auc

In [43]:
import numpy as np
import pandas as pd
#Convert a collection of raw documents to a matrix of TF-IDF features.
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import regex
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import re
import string

In [44]:
#---------------------------------------------------- Latin and Punctuation Removal-------------------------------------------
# This function will: 1- Remove latin words 2- Remove punctuations
# Source: https://stackoverflow.com/questions/18429143/strip-punctuation-with-regex-python/50985687
def latinRemoval(content):
    only_persian_list =[]
    temp_set = []

    latin = re.compile(r'[a-zA-Z0-9@$!%*?&-_`.+#(»)«,:;،؛…..؟]+')  

    for item in range(len(content)):
        for word in content[item]:
            temp_word = word.replace('\u200c', ' ', 1000000000)
               
            if not(latin.match(temp_word)) :
                if not temp_word.isdigit():
                        #source: https://pypi.org/project/persianutils/
                    word = re.findall(r'[^۰-۹]+', temp_word.strip(string.punctuation))
                    
                    if len(word) >0:    
                        processed_text = pu.standardize4Word2vec(word[0].strip())

                        if (processed_text != ' '):
#                             print(processed_text)
                            word = re.findall('\d+', processed_text.strip())
                            if len(word)==0 : 
                                temp_set.append(processed_text.strip())
                                # print (processed_text.strip())
           
        only_persian_list.append(set(temp_set))
        temp_set = []
        
    return only_persian_list

In [45]:
#--------------------------------------------------Loading StopWords ------------------------------------------
#Source of file: https://sites.google.com/site/kevinbouge/stopwords-lists
def getStopWord ():
    try:
        file = open('/content/drive/MyDrive/Final/stopwords_fa.txt', 'r', encoding='utf-8-sig')
        file_readed = file.read()
    
    finally:
        file.close()
        
    stopWord_Set = set(file_readed.split())
    return stopWord_Set


In [46]:
#------------------------------------Tokenization and Stop Word Removal------------------------------------------------------
# This function will: 1- Tokenize words  2- Remove stop words and return a list of sets
def getNoStopWord(content):
    no_StopWord_list =[]
    temp_set = []
    stop_word_list = getStopWord()
    for item in range(len(content)):
        for word in content[item]:
            if not word in stop_word_list:
                temp_set.append(word.replace('\u200c', ' ', 1000000000))
                
        no_StopWord_list.append(set (temp_set))
        temp_set = []
        
    return no_StopWord_list

In [47]:
#----------------------------------------------Lemmatization ------------------------------------------------------------------
def Lemmatization (content):
    lemmatizer = Lemmatizer()
    lemmatize_list = []
    temp_set = []
    for item in range(len(content)):
        for word in (content[item]):
            temp_set.append(lemmatizer.lemmatize(word.replace('\u200c', ' ', 1000000000)))
        lemmatize_list.append(set(temp_set))
        temp_set = []
    
    return lemmatize_list

In [48]:
#---------------------------------------------------Tokenization-------------------------------------------------------------
# Tokenizing X and at the end append Y as a label
def tokenization (X):
    tokenized_train_data = []
    for item in range(len(X)):
        var = word_tokenize(X[item])
#         var.append(Y[item])
        tokenized_train_data.append(var)
        
    return tokenized_train_data   

In [153]:
#Loading sample Dataset
#Here is a training dataset
dataFarme_train = pd.read_csv ('/content/drive/MyDrive/temp_QandA/SampleDataSet_Train_V0.csv', encoding='utf-8-sig')


Y= dataFarme_train.label
X = dataFarme_train.question  + " "+  dataFarme_train.answer

In [154]:
print(type(X))

<class 'pandas.core.series.Series'>


In [155]:
#Tokenization
tokenize_list = tokenization (X)

In [156]:
only_persian_list = latinRemoval(tokenize_list)

In [157]:
noStopWord_List = getNoStopWord(only_persian_list)

In [158]:
lemmatize_list = Lemmatization (noStopWord_List)
print (type(lemmatize_list))

<class 'list'>


In [159]:
def convertSequenceWordsToText(input_list,  fileName, lastFileName):
    X_and_Y = pd.read_csv(fileName , encoding="utf-8")
    text = ''
    counter = 0
    updator = 0
    for item in range(len(input_list)):
      for i in range(len(input_list [item + updator])):
        text += list(input_list[item + updator])[i] + ' '
      
      
      print(counter)    
      X_and_Y.loc [counter, 'text'] = text.strip()
      X_and_Y.to_csv(lastFileName ,index=False, encoding='utf-8-sig')
      counter = counter + 1
      text =''
    print("This is counter: " ,counter-1)

In [160]:
import pickle

# define a list of places

file_name = "/content/drive/MyDrive/temp_QandA/lemmatize_list.pkl"
open_file = open(file_name, "wb")
pickle.dump(lemmatize_list, open_file)
open_file.close()

file_name = "/content/drive/MyDrive/temp_QandA/lemmatize_list.pkl"
open_file = open(file_name, "rb")
lemmatize_list = pickle.load(open_file)
open_file.close()


In [161]:
convertSequenceWordsToText(lemmatize_list,'/content/drive/MyDrive/temp_QandA/SampleDataSet_Train_V0.csv', '/content/drive/MyDrive/temp_QandA/Prepared_Data.csv')

0
1
2
3
4
5
6
7
8
9
10
11
This is counter:  11


In [162]:
dataFarme_train = pd.read_csv ('/content/drive/MyDrive/temp_QandA/Prepared_Data.csv', encoding='utf-8-sig')

In [163]:
Y= dataFarme_train.label
X = dataFarme_train.text

In [201]:
#converting words to vectors
max_len = 10 
tokenizer = Tokenizer(num_words = max_len, lower = True, split = ' ')
tokenizer.fit_on_texts(X) 


X_tokenize = tokenizer.texts_to_sequences(X)

#pad sequence is used to ensure that all sequences in a list have the same length.
X_ready = pad_sequences(sequences = X_tokenize, maxlen = max_len, padding = 'pre')

In [202]:
def createLstmModel(units = 100, dropout = 0.5  ):
    
    
   #https://www.tensorflow.org/tutorials/text/text_classification_rnn
        
    #Adding Layers
    model = Sequential()
    model.add(Embedding(100 , 128 ))#max_feature = 5000 , embedding_dimention = 128

    model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2))  
   
    # model.add(LSTM(units = units , dropout = 0.2, return_sequences=True ))
    # model.add(Dense(units = units, activation="relu"))
    # model.add(Dropout(rate = dropout))


  
    model.add(LSTM(units = units , dropout = 0.2))
    model.add(Dense(units = units, activation="relu"))
    model.add(Dropout(rate = dropout))

    model.add(Dense(units = 1 ,activation="sigmoid"))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=[ 'accuracy'])
    

    return model


In [203]:
model = createLstmModel()

model.fit(X_ready, Y,  epochs=20,batch_size = 4)

Epoch 1/20
3/3 [==============================] - 0s 5ms/step - loss: 0.6948 - accuracy: 0.4167
Epoch 2/20
3/3 [==============================] - 0s 4ms/step - loss: 0.6988 - accuracy: 0.5000
Epoch 3/20
3/3 [==============================] - 0s 4ms/step - loss: 0.6946 - accuracy: 0.5833
Epoch 4/20
3/3 [==============================] - 0s 4ms/step - loss: 0.6854 - accuracy: 0.5833
Epoch 5/20
3/3 [==============================] - 0s 5ms/step - loss: 0.6778 - accuracy: 0.5833
Epoch 6/20
3/3 [==============================] - 0s 5ms/step - loss: 0.6681 - accuracy: 0.5833
Epoch 7/20
3/3 [==============================] - 0s 5ms/step - loss: 0.6533 - accuracy: 0.5833
Epoch 8/20
3/3 [==============================] - 0s 4ms/step - loss: 0.6377 - accuracy: 0.5833
Epoch 9/20
3/3 [==============================] - 0s 4ms/step - loss: 0.5991 - accuracy: 0.5833
Epoch 10/20
3/3 [==============================] - 0s 4ms/step - loss: 0.5820 - accuracy: 0.7500
Epoch 11/20
3/3 [======================

In [204]:
#Here is Testing

In [205]:
#Loading sample Dataset
#Here is a training dataset
dataFarme_test = pd.read_csv ('/content/drive/MyDrive/temp_QandA/SampleDataSet_Test.csv', encoding='utf-8-sig')


Y_test= dataFarme_test.label
X_test = dataFarme_test.question  + " "+  dataFarme_test.answet

In [206]:
tokenize_list_test = tokenization (X_test)

In [207]:
only_persian_list_test = latinRemoval(tokenize_list_test)

In [208]:
noStopWord_List_test = getNoStopWord(only_persian_list_test)

In [209]:
lemmatize_list_test = Lemmatization (noStopWord_List_test)

In [210]:
import pickle

# define a list of places

file_name = "/content/drive/MyDrive/temp_QandA/lemmatize_list_test.pkl"
open_file = open(file_name, "wb")
pickle.dump(lemmatize_list_test, open_file)
open_file.close()

file_name = "/content/drive/MyDrive/temp_QandA/lemmatize_list_test.pkl"
open_file = open(file_name, "rb")
lemmatize_list_test = pickle.load(open_file)
open_file.close()

In [211]:
convertSequenceWordsToText(lemmatize_list_test,'/content/drive/MyDrive/temp_QandA/SampleDataSet_Test.csv', '/content/drive/MyDrive/temp_QandA/Prepared_Data_test.csv')

0
1
2
3
This is counter:  3


In [212]:
dataFarme_test = pd.read_csv ('/content/drive/MyDrive/temp_QandA/Prepared_Data.csv', encoding='utf-8-sig')
Y_test= dataFarme_test.label
X_test = dataFarme_test.text

In [213]:
#converting words to vectors
tokenizer.fit_on_texts(X_test )

X_temp = tokenizer.texts_to_sequences(X_test)

#pad sequence is used to ensure that all sequences in a list have the same length.
X_ready_test = pad_sequences(sequences = X_temp, maxlen = max_len, padding = 'pre')

In [214]:
loss, accuracy = model.evaluate(X_ready_test, Y_test, verbose=0)
print('Accuracy:  %.3f' , accuracy)



Accuracy:  %.3f 0.9166666865348816


In [215]:
#THRILLING part

In [251]:
#Write your Question here ? greeting or music
Question = "امروز، حالت خوبه؟"
# Question= "موسیقی چی گوش میدی؟"
tokenize_list_test1 =tokenization (pd.Series(Question))
only_persian_list_test1 = latinRemoval(tokenize_list_test1)
noStopWord_List_test1 = getNoStopWord(only_persian_list_test1)
lemmatize_list_test1 = Lemmatization (noStopWord_List_test1)

In [252]:
print(lemmatize_list_test1)

[{'حالت', 'خوبه'}]


In [253]:

temp_var = ""
for item in range(len(lemmatize_list_test1)):
  for i in range(len(lemmatize_list_test1[item])):
    temp_var += list (lemmatize_list_test1[item])[i] + " "

In [254]:
print(temp_var)

حالت خوبه 


In [255]:
#converting words to vectors

tokenizer.fit_on_texts(temp_var )

X_temp_test1 = tokenizer.texts_to_sequences(temp_var)
print(X_temp_test1)
#pad sequence is used to ensure that all sequences in a list have the same length.
X_ready_test1 = pad_sequences(sequences = X_temp_test1, maxlen = max_len, padding = 'pre')

[[3], [4], [5], [1], [], [6], [2], [7], [8], []]


In [256]:
y_predict =model.predict_classes(X_ready_test1)

In [257]:
print( y_predict)

[[0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]]


In [258]:
count_0 = 0
count_1= 0
for i in y_predict:
  if i == 0:
    count_0 +=1
  else: 
    count_1 +=1


if count_1 > count_0:
  print(1)
else:
    print(0)




0
